In [1]:
import gensim
import re
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from torch.autograd import Variable

import torch
import torch.nn as nn

import nltk
nltk.download('stopwords')

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/zhipeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [3]:
def text_prepare(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = GOOD_SYMBOLS_RE.sub('', text)
    text = ' '.join([x for x in text.split() if x and x not in STOPWORDS])
    return text.strip()

In [4]:
starspace_embeddings = {}
for line in open('./starspace_embeddings/mdl.tsv'):
    try:
      word, *vec = line.strip().split()
      vf = []
      for v in vec:
          vf.append(float(v))
      starspace_embeddings[word] = np.array(vf)
    except:
      # print(word)
      # print(*vec)
      continue

In [5]:
print(type(starspace_embeddings), len(starspace_embeddings))

<class 'dict'> 295698


In [6]:
def query_to_vec(query, embeddings, dim=100):
  """
  """
  vec = np.zeros((dim,), dtype=np.float32)
  count = 0
  for w in query.split():
    if w in embeddings:
      count += 1
      vec += embeddings[w]
  if count == 0:
    return vec
  return vec/count

In [7]:
def read_data(data_file):
  
  X = []
  Y = []
  
  with open(data_file, 'r') as data_file:
    for line in data_file:
      pid, question, prediction, answer, label = line.split('\t')

      question = text_prepare(question)
      answer = text_prepare(answer)
      
      prediction = text_prepare(prediction)
      prediction = [w for w in prediction.split() if w in starspace_embeddings]
      prediction = ' '.join(prediction)
        
      if len(question.split()) > 16 or len(question.split()) < 3:
        continue
      if len(answer.split()) > 128 or len(answer.split()) < 5:
        continue
      
      # question = [w for w in question.split() if w in starspace_embeddings]
      # answer = [w for w in answer.split() if w in starspace_embeddings]
      
      # question = " ".join(question)
      # answer = " ".join(answer)
      
      question = question + " " + prediction
      qa_pair = (question, answer)
      label = int(label)
      X.append(qa_pair)
      Y.append(label + 2)
  
  X, Y = shuffle(X, Y, random_state=0)
  X = np.asarray(X)
  Y = np.asarray(Y)
  
  return X, Y

In [8]:
X_train, Y_train = read_data('./question2comment/train_expansion_4cls')
X_val, Y_val = read_data('./question2comment/validate_expansion_4cls')
print(type(X_train), X_train.shape )
print(type(Y_train), Y_train.shape )
print()
print(type(X_val), X_val.shape )
print(type(Y_val), Y_val.shape )

<class 'numpy.ndarray'> (350522, 2)
<class 'numpy.ndarray'> (350522,)

<class 'numpy.ndarray'> (42443, 2)
<class 'numpy.ndarray'> (42443,)


In [9]:
print(set(Y_train))

{0, 1, 2, 3}


In [10]:
question_maxlen = max( [len(qapair[0].split()) for qapair in X_val] )
print(question_maxlen, type(question_maxlen))
answer_maxlen = max( [len(qapair[1].split()) for qapair in X_val] )
print(answer_maxlen, type(answer_maxlen))

28 <class 'int'>
128 <class 'int'>


In [11]:
Xqtrain = X_train[:, 0]
Xatrain = X_train[:, 1]

Xqval = X_val[:, 0]
Xaval = X_val[:, 1]

In [12]:
print("Training Set:")
print(type(Xqtrain), Xqtrain.shape)
print(type(Xatrain), Xatrain.shape)
print(type(Y_train), Y_train.shape)
print("Validation Set:")
print(type(Xqval), Xqval.shape)
print(type(Xaval), Xaval.shape)
print(type(Y_val), Y_val.shape)

Training Set:
<class 'numpy.ndarray'> (350522,)
<class 'numpy.ndarray'> (350522,)
<class 'numpy.ndarray'> (350522,)
Validation Set:
<class 'numpy.ndarray'> (42443,)
<class 'numpy.ndarray'> (42443,)
<class 'numpy.ndarray'> (42443,)


In [13]:
# build vocab 
vocab = []

for e in Xqtrain:
  for word in e.split():
    vocab.append(word)

for e in Xatrain:
  for word in e.split():
    vocab.append(word)

vocab = set(vocab)

In [14]:
word2index={'pad': 0, 'unk': 1}

for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
        
index2word = {v:k for k, v in word2index.items()}

target2index = {}

for cl in set(Y_train):
  if target2index.get(cl) is None:
    target2index[cl] = len(target2index)

    
index2target = {v:k for k, v in target2index.items()}
vocab_size = len(word2index)
print(vocab_size)

122960


In [16]:
import pickle
with open("./Model/word2index.pkl", "wb") as handler:
    pickle.dump(word2index, handler)

In [17]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [18]:
def prepare_sequence(seq, to_index):
    # idxs = list(map(lambda w: to_index[w] if to_index.get(w) is not None else to_index["<UNK>"], seq))
    idxs = []
    for w in seq.split():
      if w in word2index:
        idxs.append(word2index[w])
      else:
        idxs.append(word2index["unk"])
    # return Variable(LongTensor(idxs))
    return idxs

In [19]:
Xq_p, Xa_p = [], []
for xq, xa in zip(Xqtrain, Xatrain):
  Xq_p.append(prepare_sequence(xq, word2index))
  Xa_p.append(prepare_sequence(xa, word2index))
Xq_train = pad_sequences(Xq_p, maxlen=16)
Xa_train = pad_sequences(Xa_p, maxlen=128)

Xq_p, Xa_p = [], []
for xq, xa in zip(Xqval, Xaval):
  Xq_p.append(prepare_sequence(xq, word2index))
  Xa_p.append(prepare_sequence(xa, word2index))
Xq_val = pad_sequences(Xq_p, maxlen=16)
Xa_val = pad_sequences(Xa_p, maxlen=128)

In [20]:
print( type(Xq_train), Xq_train.shape, Xq_train[0] )
print( type(Xa_train), Xa_train.shape, Xa_train[0] )

print( type(Xq_val), Xq_val.shape, Xq_val[0] )
print( type(Xa_val), Xa_val.shape, Xa_val[0] )

<class 'numpy.ndarray'> (350522, 16) [     0      0  28442  23627  91315  52878   7215  76085  68177  20505
  94237 107978  38333  23323  36488  32533]
<class 'numpy.ndarray'> (350522, 128) [     0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0 

In [21]:
vocab_size = len(word2index)
print(vocab_size)
EMBEDDING_SIZE = 100

122960


In [22]:
# import gensim
# from gensim.models import FastText
# fasttext_model = FastText.load("./fasttext_embeddings/fasttext.model")
'''
embedding_size = 100

pretrained = []
zero_embeddings = np.zeros((embedding_size,))
count = 0

for key in word2index.keys():
    if key in fasttext_model.wv.vocab:
        count += 1
        pretrained.append( fasttext_model.wv[key] )
    else:
        pretrained.append( fasttext_model.wv["unk"] )

print(count)
pretrained_vectors = np.vstack(pretrained)
'''

'\nembedding_size = 100\n\npretrained = []\nzero_embeddings = np.zeros((embedding_size,))\ncount = 0\n\nfor key in word2index.keys():\n    if key in fasttext_model.wv.vocab:\n        count += 1\n        pretrained.append( fasttext_model.wv[key] )\n    else:\n        pretrained.append( fasttext_model.wv["unk"] )\n\nprint(count)\npretrained_vectors = np.vstack(pretrained)\n'

In [23]:
# fasttext_model.wv["unk"]

In [24]:
embeddings = starspace_embeddings
embedding_size = 100

pretrained = []
zero_embeddings = np.zeros((embedding_size,))
count = 0

for key in word2index.keys():
  # print(key)
  if key in embeddings:
    count += 1
    pretrained.append( embeddings[key] )
  else:
    pretrained.append( np.random.randn(100) )

print(count)
pretrained_vectors = np.vstack(pretrained)

110974


In [25]:
print( type(pretrained_vectors) , pretrained_vectors.shape)

<class 'numpy.ndarray'> (122960, 100)


In [26]:
pretrained_vectors

array([[-5.74498e-03, -1.39079e-02,  1.18770e-01, ..., -7.53179e-02,
         3.80801e-02,  5.68262e-02],
       [-4.98624e-02, -2.49910e-02, -1.71854e-02, ...,  1.73683e-02,
         3.90429e-03, -2.50023e-02],
       [ 3.93177e-02,  2.93438e-02,  1.96626e-03, ...,  3.86291e-03,
         4.31270e-02,  6.19399e-02],
       ...,
       [ 5.61106e-02, -2.46822e-02,  5.14951e-02, ..., -2.67433e-02,
        -5.53501e-02, -3.01738e-02],
       [-5.45331e-02, -1.30692e-03,  4.95738e-02, ..., -1.53982e-02,
         5.44395e-02,  2.93648e-02],
       [-1.92501e-04, -1.00505e-04, -5.68038e-04, ...,  1.58721e-03,
         1.06467e-03, -1.40518e-03]])

##  Building Modle

In [27]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import nltk
import re
from copy import deepcopy
flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)
torch.manual_seed(777)  # reproducibility

In [28]:
USE_CUDA = torch.cuda.is_available()
gpus = [0]
torch.cuda.set_device(gpus[0])

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [29]:
USE_CUDA

True

In [30]:
class Encoder(nn.Module):
  
  def __init__(self, vocab_size, embedding_dim, output_size, kernel_dim=100, kernel_sizes=(3, 4, 5), dropout=0.5):
    
    super(Encoder,self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.convs = nn.ModuleList([nn.Conv2d(1, kernel_dim, (K, embedding_dim)) for K in kernel_sizes])
    
    # kernal_size = (K,D) 
    self.dropout = nn.Dropout(dropout)
    # self.fc = nn.Linear(len(kernel_sizes) * kernel_dim, output_size)
    self.linear1 = torch.nn.Linear(600, 1028, bias=True)
    self.linear2 = torch.nn.Linear(1028, 128, bias=True)
    self.linear3 = torch.nn.Linear(128, 4, bias=True)
    self.linear4 = torch.nn.Linear(4, 4, bias=True)
    
    # torch.nn.init.xavier_uniform(self.linear1.weight)
    self.relu = torch.nn.ReLU()
    self.dropout = torch.nn.Dropout(p=1 - keep_prob)
    
  def init_weights(self, pretrained_word_vectors, is_static=True):
    torch.nn.init.xavier_uniform(self.linear1.weight)
    torch.nn.init.xavier_uniform(self.linear2.weight)
    torch.nn.init.xavier_uniform(self.linear3.weight)
    torch.nn.init.xavier_uniform(self.linear4.weight)
    self.embedding.weight = nn.Parameter(torch.from_numpy(pretrained_word_vectors).float())
    # self.embedding.weight.requires_grad = False
    if is_static:
      self.embedding.weight.requires_grad = False
    
    
  def encode(self, inputs):
    inputs = self.embedding(inputs).unsqueeze(1) # (B,1,T,D)
    inputs = [F.relu(conv(inputs)).squeeze(3) for conv in self.convs] 
    inputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in inputs]
    concated = torch.cat(inputs, 1)
    concated = self.dropout(concated)
    return concated
  
  
  def forward(self, xq, xa, is_training=False):
    
    encode_xq = self.encode(xq)
    encode_xa = self.encode(xa)
    
    
    x = torch.cat((encode_xq, encode_xa), 1)
    # print(x.shape)
    
    x = self.linear1(x)
    x = self.relu(x)
    x = self.dropout(x)
    
    x = self.linear2(x)
    x = self.relu(x)
    x = self.dropout(x)
    
    x = self.linear3(x)
    # x = self.linear4(x)
    # print(type(inputs), inputs.shape)
    # print(inputs)
    # print( self.embedding(inputs) )
    # inputs = self.embedding(inputs).unsqueeze(1) # (B,1,T,D)
    # inputs = [F.relu(conv(inputs)).squeeze(3) for conv in self.convs] 
    # inputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in inputs]
    # concated = torch.cat(inputs, 1)
    # print("concated")
    # print(concated, concated.shape)
    # if is_training:
    # concated = self.dropout(concated)
    
    return x

In [31]:
EPOCH = 5
KERNEL_SIZES = [3,4,5]
KERNEL_DIM = 100
LR = 0.001
BATCH_SIZE = 256
keep_prob = 0.5

In [32]:
encoder = Encoder(len(word2index), 100, 3, KERNEL_DIM, KERNEL_SIZES)
encoder.init_weights(pretrained_vectors) # initialize embedding matrix using pretrained vectors
# net = Net(encoder)

/home/zhipeng/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/zhipeng/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/zhipeng/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/zhipeng/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [33]:
if USE_CUDA:
    encoder = encoder.cuda()
    # net = net.cuda()
    
loss_function = nn.CrossEntropyLoss()
# params = list(net.parameters()) + list(encoder.parameters())
optimizer = optim.Adam(encoder.parameters(), lr=LR)

In [34]:
train_question_inps = torch.from_numpy(Xq_train)
train_question_inps = train_question_inps.type(torch.LongTensor).cuda()

train_answer_inps = torch.from_numpy(Xa_train)
train_answer_inps = train_answer_inps.type(torch.LongTensor).cuda()

train_tgts = torch.from_numpy(Y_train)
train_tgts = train_tgts.type(torch.LongTensor).cuda()


val_question_inps = torch.from_numpy(Xq_val)[:5000]
val_question_inps = val_question_inps.type(torch.LongTensor).cuda()

val_answer_inps = torch.from_numpy(Xa_val)[:5000]
val_answer_inps = val_answer_inps.type(torch.LongTensor).cuda()

val_tgts = torch.from_numpy(Y_val)[:5000]
val_tgts = val_tgts.type(torch.LongTensor).cuda()

In [35]:
print(type(train_question_inps), train_question_inps.shape)
print(type(train_answer_inps), train_answer_inps.shape)
print(type(train_tgts), train_tgts.shape)

print(type(val_question_inps), val_question_inps.shape)
print(type(val_answer_inps), val_answer_inps.shape)
print(type(val_tgts), val_tgts.shape)

<class 'torch.Tensor'> torch.Size([350522, 16])
<class 'torch.Tensor'> torch.Size([350522, 128])
<class 'torch.Tensor'> torch.Size([350522])
<class 'torch.Tensor'> torch.Size([5000, 16])
<class 'torch.Tensor'> torch.Size([5000, 128])
<class 'torch.Tensor'> torch.Size([5000])


In [36]:
import torch.utils.data as data_utils
train = data_utils.TensorDataset(train_question_inps, train_answer_inps, train_tgts)
train_loader = data_utils.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

In [139]:
EPOCH = 5
encoder = encoder.train()  # explicitly set

for epoch in range(EPOCH):
  avg_cost = 0
  total_batch = len(train) // BATCH_SIZE
  # losses = []
  for i, (batch_xqs, batch_xas, batch_ys) in enumerate(train_loader):
    Xq = Variable(batch_xqs)
    Xa = Variable(batch_xas)
    Y = Variable(batch_ys)
    
    # print(type(Xq), Xq.shape)
    # print(type(Xa), Xa.shape)
    # print(type(Y), Y.shape)
    
    # net.zero_grad()
    encoder.zero_grad()
    preds = encoder(Xq, Xa)
    
    # print("preditions", preds)
    # print("gt", Y)
    
    
    cost = loss_function(preds, Y)
    cost.backward()
    
    # for param in net.parameters():
    #   param.grad.data.clamp_(-3, 3)
    
    # for param in encoder.parameters():
    #   param.grad.data.clamp_(-3, 3)
    
    optimizer.step()
    avg_cost += cost / total_batch
    
    if i % 500 == 0:
      correct_prediction = (torch.max(preds.data, 1)[1] == Y.data)
      accuracy = correct_prediction.float().mean()
      print('Training Accuracy:', accuracy, cost)
    
    # break
  
  print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost.data))
  
  Xq_val = Variable(val_question_inps) 
  Xa_val = Variable(val_answer_inps)
  Y_val = Variable(val_tgts)
  val_preds = encoder(Xq_val, Xa_val)
  correct_prediction = (torch.max(val_preds.data, 1)[1] == Y_val.data)
  accuracy = correct_prediction.float().mean()
  print('Validation Accuracy:', accuracy, cost)
  
  # break
  # del Xq_val, Xa_val, Y_val, val_preds
  
print('Learning Finished!')

Training Accuracy: tensor(0.4766, device='cuda:0') tensor(1.0618, device='cuda:0', grad_fn=<NllLossBackward>)
Training Accuracy: tensor(0.5000, device='cuda:0') tensor(1.0424, device='cuda:0', grad_fn=<NllLossBackward>)
Training Accuracy: tensor(0.5156, device='cuda:0') tensor(1.0690, device='cuda:0', grad_fn=<NllLossBackward>)
[Epoch:    1] cost = 1.05310094
Validation Accuracy: tensor(0.4594, device='cuda:0') tensor(1.0145, device='cuda:0', grad_fn=<NllLossBackward>)
Training Accuracy: tensor(0.5430, device='cuda:0') tensor(1.0963, device='cuda:0', grad_fn=<NllLossBackward>)
Training Accuracy: tensor(0.5117, device='cuda:0') tensor(1.0855, device='cuda:0', grad_fn=<NllLossBackward>)
Training Accuracy: tensor(0.5586, device='cuda:0') tensor(0.9999, device='cuda:0', grad_fn=<NllLossBackward>)
[Epoch:    2] cost = 1.0525949
Validation Accuracy: tensor(0.4628, device='cuda:0') tensor(1.0647, device='cuda:0', grad_fn=<NllLossBackward>)
Training Accuracy: tensor(0.5156, device='cuda:0') te

# Evaluation

In [140]:
def hits_count(candidate_ranks, k):
  '''
    candidate_ranks:
    list of candidates' ranks; one rank per question;
    length is a number of questions
    rank is a number from q to len(candidates of the question)
    e.g. [2, 3] means that first candidate has the rank 2,
                           second candidate has the rank 3
    k: number of top-ranked elements (k in hits@k metric)
    result: return Hits@k value for current ranking 
  '''
  count = 0
  for rank in candidate_ranks:
    if rank <= k:
      count += 1
  return count/(len(candidate_ranks)+1e-8)

In [141]:
def dcg_score(candidate_ranks, k):
  '''
    candidate_ranks:
    list of candidates' ranks; one rank per question;
    length is a number of questions
    rank is a number from q to len(candidates of the question)
    e.g. [2, 3] means that first candidate has the rank 2,
                           second candidate has the rank 3
    k: number of top-ranked elements (k in hits@k metric)
    
    result: return DCG@k value for current ranking
  '''
  score = 0
  for rank in candidate_ranks:
    if rank <= k:
      score += 1/np.log2(1+rank)
  return score/(len(candidate_ranks)+1e-8)

In [143]:
evaluation_qa_pairs = []
with open('./evaluation_data', 'r') as evaluation_data:
  for line in evaluation_data:
    qa = line.strip().split('\t')
    # print(len(qa))
    # print(qa)
    question = qa[0]
    prediction = qa[1] 
    question = question + ' '.join(set(text_prepare(prediction).split()))
    # print(question)
    candidate_answers = qa[2:7]
    
    # print(prediction)
    # print(candidate_answers)
    # break
    evaluation_qa_pairs.append( (question, candidate_answers) )

In [145]:
# load model 
encoder.eval()    # set the model to evaluation mode (dropout=False)

Encoder(
  (embedding): Embedding(122960, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (linear1): Linear(in_features=600, out_features=1028, bias=True)
  (linear2): Linear(in_features=1028, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=4, bias=True)
  (linear4): Linear(in_features=4, out_features=4, bias=True)
  (relu): ReLU()
)

In [146]:
def rank_candidates(question, candidate_answers, model):
  '''
    question: a string
    candidate_answers : a list of strings
    
    result: a list of pairs (initial position in the list, question)
  '''
  candidate_scores = []
  for answer in candidate_answers:
    # question_vec = query_to_vec(question, starspace_embeddings, dim=100)
    # answer_vec = query_to_vec(answer, starspace_embeddings, dim=100)
    
    Xq_p, Xa_p = [], []
    Xq_p.append(prepare_sequence(question, word2index))
    Xa_p.append(prepare_sequence(answer, word2index))
    
    # print(Xq_p)
    # print(Xa_p)
    
    Xq_val = torch.from_numpy( pad_sequences(Xq_p, maxlen=16)  )
    Xq_val = Xq_val.type(torch.LongTensor).cuda()
    
    Xa_val = torch.from_numpy( pad_sequences(Xa_p, maxlen=128) )
    Xa_val = Xa_val.type(torch.LongTensor).cuda()
    
    
    Xq_val = Variable(Xq_val)
    Xa_val = Variable(Xa_val)
    
    preds = encoder(Xq_val, Xa_val)
    preds = F.softmax( preds ) 
    
    # print( type(preds), preds.shape, preds)
    
    np_prediction = preds.detach().cpu().numpy()
    # print(np_prediction, np_prediction[0][-1])
    ########################################
    ######## Check here !!! Important ######
    ########################################
    score = (-2.*np_prediction[0][0]) + (-1.*np_prediction[0][1]) + (1.*np_prediction[0][2]) + (2.*np_prediction[0][3])
    candidate_scores.append( score )
    # candidate_scores.append( np_prediction[0][-1] )
 
    
  tl = [(i, candidate_answers[i], candidate_scores[i]) for i in range(len(candidate_answers))]
  # print(tl)
  stl = sorted(tl, key=lambda x:x[2], reverse=True)
  result = [(t[0],t[1]) for t in stl]
  return result

In [147]:
# load model 
encoder.eval()    # set the model to evaluation mode (dropout=False)
# Test model and check accuracy
model_ranking = []
model = encoder

for qa_pair in evaluation_qa_pairs:

  question = qa_pair[0]
  candidate_answers = qa_pair[1]
  
  ranks = rank_candidates(question, candidate_answers, model)
  # print(ranks)
  model_ranking.append( [r[0] for r in ranks].index(0) + 1 )
  # break
  # print(ranks)
  # model_ranking.append( [r[0] for r in ranks].index(0) + 1 )
  # break

/home/zhipeng/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [148]:
for k in [1, 2, 3, 4, 5]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(model_ranking, k), 
                                               k, hits_count(model_ranking, k)))

DCG@   1: 0.400 | Hits@   1: 0.400
DCG@   2: 0.559 | Hits@   2: 0.652
DCG@   3: 0.641 | Hits@   3: 0.817
DCG@   4: 0.690 | Hits@   4: 0.930
DCG@   5: 0.717 | Hits@   5: 1.000


In [128]:
torch.save(encoder, './Model/expansion-40.6.pt')